In [9]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import AnimalShelter CRUD module
from AacCrud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "TestUser@123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Jeff Fulfer\'s AnimalShelter Dashboard'))),
    html.A(
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'width': '150px',
                'height': 'auto',
                'margin-left': 'auto',
                'margin-right': 'auto',
                'display': 'block'
            },
        ),
        href="https://www.snhu.edu",
        target="_blank"
    ),
    html.Hr(),
    html.Div(
        style={'display': 'inline-block'},
        children=[
            html.Label("Select Rescue Type:"),
            dcc.Dropdown(
                id='filterOptions',
                options=[
                    {'label': 'Water Rescue', 'value': 'Water'},
                    {'label': 'Wilderness Rescue', 'value': 'Wilderness'},
                    {'label': 'Disaster Rescue', 'value': 'Disaster'},
                    {'label': 'All Types', 'value': 'All'}
                ],
                value='All',
                style={'width': '200px'}
            )
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         row_selectable='single',
                         column_selectable='single',
                         selected_rows=[0],
                         selected_columns=[],
                         page_action='native',
                         page_size=25,
                         sort_action='native',
                         filter_action='native'
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id', 'columns')],
    [Input('filterOptions', 'value')]
)
def update_dashboard(filter_type):
    
    # Create query parameters for Water Rescue type
    if (filter_type == 'Water'):
        #print("Filter type: Water")
        searchParams = {
            "breed": {
                "$in": [
                    "Labrador Retriever Mix",
                    "Chesapeake Bay Retriever",
                    "Newfoundland"
                ]
            },
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156
            }
        }
        
    # Create query paramters for Wilderness Rescue Type
    elif (filter_type == 'Wilderness'):
        #print("Filter type: Wilderness")
        searchParams = {
            "breed": {
                "$in": [
                    "German Shepherd",
                    "Alaskan Malamute",
                    "Old English Sheepdog",
                    "Siberian Huskey",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 26,
                "$lte": 156
            }
        }
        
    # Create query parameters for Disaster Rescue Type
    elif (filter_type == 'Disaster'):
        #print("Filter type: Disaster")
        searchParams = {
            "breed": {
                "$in": [
                    "Doberman Pinscher",
                    "German Shepherd",
                    "Golden Retriever",
                    "Bloodhound",
                    "Rottweiler"
                ]
            },
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {
                "$gte": 20,
                "$lte": 300
            }
        }
        
    # Used as default All Types option to search for all types
    else:
        searchParams = {}
        
    #print("Params: ", searchParams)
        
    # Gather results
    results = db.read(searchParams)
    
    # Convert to DataFrame
    df = pd.DataFrame.from_records(results)
    
    # Drop _id column
    if ('_id' in df.columns):
        df.drop(columns=['_id'], inplace=True)
    
    # Define column headers and options
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    
    # Prepare data for DataTable
    data=df.to_dict('records')
    
    # Return prepared data and columns
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('filterOptions', 'value')]
)
def update_graphs(viewData, filter_type):
    
    # Convert to DataFrame
    df = pd.DataFrame(viewData)
    
    # Check for empty DataFrame or missing breed column
    if df.empty or 'breed' not in df.columns:
        return [html.P("No 'breed' data available.")]
    
    # Count breed frequency
    breedCount = df['breed'].value_counts().reset_index()
    breedCount.columns = ['breed', 'count']
    
    # Check if all filter is used and limit to top ten breeds
    if (filter_type == 'All'):
        breedCount = breedCount.head(10)
    
    # Create Pie Chart
    fig = px.pie(breedCount, names='breed', values='count',
                title='Animal Breed Count')
    

    return [
        dcc.Graph(            
            figure = fig
        )    
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
        
    # Austin, TX is at [30.75,-97.48]
        
    # Receiving an index out of bounds error when initially generating map.
    # Returning empty map if row does not have enough columns or if either 
    # cells are empty
    #
    if (
        dff.empty or
        dff.shape[1] < 15 or
        row >= len(dff) or
        pd.isna(dff.iloc[row, 13]) or
        pd.isna(dff.iloc[row, 14])
    ):
        return dl.Map(style={'width': '1000px', 'height': '500px'},
                     center=[30.75,-97.98], zoom=10, children=[
                         dl.TileLayer(id='base-layer-id')
                     ])
    
    # Return position on map if positional data is available
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
              center=[30.75,-97.98], zoom=10, children=[
                  dl.TileLayer(id='base-layer-id'),
                  # Marker with tool tip and pop up
                  #
                  # Column 13 & 14 define grid-coordinates
                  # Column 4 defines the breed for the animal
                  # Column 9 defines the name of the animal
                  dl.Marker(position=[dff.iloc[row, 13],dff.iloc[row, 14]],
                           children=[
                               dl.Tooltip(dff.iloc[row, 9]),
                               dl.Popup([
                                   html.H2("Animal Name:"),
                                   html.P(dff.iloc[row, 9]),
                                   html.H2("Breed:"),
                                   html.P(dff.iloc[row, 4])
                               ])
                           ])
              ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:17243/
